In [360]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 31 # Which month and days result you want to visualize in table
month = 10
repetitionValue = 24
repetitionName = "udrlBC2"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

27
3.0


In [361]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 234)
(10, 234)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11924\489148831.py:172: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,0,1,2,6,7,8,12,13,14,18,19,20,21,22,23,24,25,26,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,69,70,71,75,76,77,78,81,82,83,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,120,121,122,123,124,125,126,127,128,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.758333,0.741667,0.733333,0.625,0.641667,0.658333,0.683333,0.666667,0.666667,None,0.552778,0.602778,0.552778,0.663889,0.713889,None,None,None,0.758333,0.758333,0.733333,0.691667,0.716667,0.716667,0.733333,0.733333,0.725,0.7,0.708333,0.716667,0.758333,0.758333,0.725,0.661111,0.661111,0.669444,0.786111,0.730556,0.736111,0.741667,0.744444,0.675,0.75,0.741667,0.733333,0.75,0.758333,0.716667,0.625,0.658333,0.658333,0.691667,0.666667,0.666667,None,0.566667,0.619444,None,None,None,None,0.616667,0.641667,0.65,0.675,0.7,0.691667,0.641667,0.658333,0.65,0.602778,0.630556,0.666667,0.605556,0.691667,0.652778,0.65,0.655556,0.675,0.633333,0.683333,0.675,0.633333,0.691667,0.675,0.7,0.666667,0.666667,None,0.541667,0.611111,0.586111,0.644444,0.688889,None,None,None,0.683333,0.666667,0.666667,0.591667,0.605556,0.636111,0.661111,0.655556,0.644444,0.675,0.647222,0.597222,0.675,0.65,0.666667,0.7,0.65,0.666667,None,0.608333,0.588889,None,None,None,None,None,None,None,0.541667,0.608333,None,None,0.548148,0.508333,0.591667,0.625,0.525,0.575,0.65,0.516667,0.575,0.625,0.588889,0.610185,0.597222,0.6,0.675,0.683333,0.633333,0.666667,0.683333,0.516667,0.575,0.625,0.575,0.658333,0.708333,None,None,None,None,None,None,0.533333,0.525,0.583333,0.6,0.591667,0.625,None,None,None,0.508333,0.591667,0.625,0.6,0.675,0.683333,None,None,None,0.525,0.575,0.65,0.633333,0.666667,0.683333,None,None,None
2,0.673611,0.729167,0.6875,0.722222,0.6875,0.659722,0.652778,0.645833,0.652778,0.675926,0.675926,0.736111,0.729167,0.634259,0.68287,0.631944,None,0.55787,0.680556,0.75,0.6875,0.6875,0.729167,0.666667,0.680556,0.729167,0.680556,0.715278,0.722222,0.680556,0.666667,0.736111,0.694444,0.722222,0.708333,0.712963,0.650463,0.650463,0.694444,0.675926,0.606481,0.715278,0.666667,0.736111,0.680556,0.666667,0.75,0.680556,0.722222,0.6875,0.666667,0.673611,0.659722,0.652778,0.645833,0.675926,0.736111,None,0.627315,None,0.576389,0.722222,0.694444,0.666667,0.729167,0.722222,0.680556,0.729167,0.6875,0.666667,0.703704,0.701389,0.696759,0.710648,0.68287,0.657407,0.685185,0.652778,0.699074,0.715278,0.715278,0.659722,0.708333,0.708333,0.659722,0.659722,0.652778,0.680556,0.631944,0.675926,0.715278,0.708333,0.655093,0.657407,0.625,None,0.576389,0.673611,0.638889,0.659722,0.699074,0.662037,0.671296,0.657407,0.618056,0.645833,0.659722,0.574074,0.675926,0.666667,0.645833,0.645833,0.659722,0.645833,0.645833,0.701389,0.6

In [362]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.79 : subject 2, feature:dataHRCV-BC--dataWCV-BC, kernellinear, C:0.5
Accuracy 0.79 : subject 1, feature:dataWCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.78 : subject 3, feature:dataHRCV-BC--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 3, feature:dataHRCV-BC--welchDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 3, feature:dataHRCV-BC--dataFFTCV-BC, kernelsigmoid, C:2.5
Accuracy 0.77 : subject 5, feature:dataCV2--fftDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:welchDataBC--fftData, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:dataCV2--welchDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:dataWCV-BC--dataHRCV-BC, kernellinear, C:0.5
Accuracy 0.77 : subject 3, feature:welchDataBC--fftData, kernelsigmoid, C:2.5


In [363]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.65203207 0.67907886 0.65221379 0.62655675 0.65088039
 0.6369984  0.61498879 0.68019713 0.60173415]


,0,1,2,6,7,8,12,13,14,18,19,20,21,22,23,24,25,26,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,69,70,71,75,76,77,78,81,82,83,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,120,121,122,123,124,125,126,127,128,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233
0,0.692925,0.69126,0.691222,0.686223,0.682978,0.675486,0.616344,0.634241,0.622409,0.607453,0.609775,0.635531,0.605859,0.635408,0.65144,0.566935,0.598709,0.569412,0.694005,0.695099,0.690381,0.693865,0.69414,0.686527,0.690147,0.690264,0.69184,0.686995,0.682739,0.677156,0.694444,0.690586,0.688286,0.673929,0.671951,0.666486,0.687868,0.693088,0.69323,0.692967,0.663571,0.669845,0.687004,0.690021,0.690123,0.694515,0.697344,0.688496,0.688215,0.691143,0.676445,0.632997,0.636195,0.628877,0.639436,0.61422,0.634772,0.555556,0.59581,0.594893,0.575793,0.684077,0.686134,0.675519,0.679031,0.688632,0.666288,0.681865,0.685859,0.673836,0.665529,0.667667,0.665471,0.668773,0.678425,0.671083,0.664283,0.651102,0.673371,0.703493,0.695581,0.678554,0.702497,0.696507,0.675397,0.640951,0.639983,0.634035,0.630015,0.614191,0.631492,0.640283,0.641138,0.66565,0.594241,0.596282,0.571979,0.618023,0.630107,0.628358,0.624334,0.626549,0.635567,0.616904,0.64393,0.63712,0.634152,0.631978,0.627081,0.628316,0.628769,0.624037,0.629775,0.628273,0.624724,0.627634,0.618245,0.622866,0.572222,0.590278,0.609028,0.59335,0.593504,0.568699,0.604965,0.61645,0.626977,0.592047,0.630404,0.631031,0.612103,0.632813,0.642363,0.61315,0.623995,0.647199,0.605303,0.62197,0.646783,0.637712,0.628437,0.630776,0.620854,0.645473,0.675047,0.613529,0.654473,0.673601,0.602988,0.631887,0.638384,0.61381,0.651389,0.658633,0.580791,0.571675,0.577125,0.578687,0.567045,0.577666,0.596727,0.618705,0.639694,0.633602,0.610685,0.637274,0.574046,0.596801,0.580051,0.612103,0.632813,0.642363,0.620854,0.645473,0.675047,0.580791,0.571675,0.577125,0.61315,0.623995,0.647199,0.613529,0.654473,0.673601,0.578687,0.567045,0.577666


Max average accuracy 0.7034932659932659
108
9.0
dataWCV-BC--fftData
sigmoid
C: 2.5


In [364]:
# x = 0
# for names in nameDict.keys():
#     print(names)
#     x +=1
#     print(x)

In [365]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,6,7,8,12,13,14,18,19,20,21,22,23,24,25,26,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,69,70,71,75,76,77,78,81,82,83,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,120,121,122,123,124,125,126,127,128,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233
0,6.236322,6.221338,6.221002,6.17601,6.146801,6.079377,5.547096,5.708165,5.601684,4.859624,4.878199,5.719781,4.846871,4.447854,5.862963,3.968547,1.796128,3.41647,6.246044,6.255892,6.213426,6.244781,6.247264,6.178746,6.211322,6.212374,6.226557,6.182955,6.144655,6.094402,6.25,6.215278,6.194571,6.065362,6.047559,5.998373,6.190811,6.237795,6.239071,6.2367,5.972138,6.028605,6.183039,6.210185,6.211111,6.250631,6.276094,6.196465,6.193939,6.220286,6.088005,5.69697,5.725758,5.659891,5.115488,4.913763,5.712949,0.555556,3.57486,1.78468,3.454756,6.156692,6.17521,6.079672,6.111279,6.197685,5.996591,6.136785,6.172727,6.06452,5.989759,6.009007,5.98924,6.018953,6.105822,6.039745,5.978549,5.859919,6.06034,6.331439,6.260227,6.106987,6.322475,6.268561,6.078577,5.768561,5.759848,5.706313,5.040123,4.913524,5.683432,5.122264,4.487963,5.325196,3.565446,1.788847,3.431874,5.562205,5.67096,5.655219,5.61901,5.638945,5.720104,5.552132,5.79537,5.734077,5.707365,5.055822,5.643729,5.65484,5.658923,5.61633,5.667971,5.654461,5.622517,5.021072,4.94596,5.605794,1.716667,0.590278,1.218056,3.560101,1.780513,3.412191,4.839717,4.931598,5.642789,4.736373,3.782421,5.048246,5.508923,5.0625,5.781271,5.51835,4.991961,5.82479,5.447727,4.975758,5.821044,5.101698,4.39906,5.046207,4.966835,4.518308,5.400379,5.521759,4.581313,5.388805,5.426894,5.055093,5.745455,4.91048,4.559722,5.927694,2.903956,1.715025,3.462753,2.893434,1.701136,3.465993,5.370539,4.330934,5.117551,5.068813,4.88548,5.09819,3.444276,1.790404,3.480303,5.508923,5.0625,5.781271,4.966835,4.518308,5.400379,2.903956,1.715025,3.462753,5.51835,4.991961,5.82479,5.521759,4.581313,5.388805,2.893434,1.701136,3.465993


108
9.0
Max total significant accuracy then averaged 0.7034932659932659 
dataWCV-BC--fftData
sigmoid
C: 2.5 


In [366]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
